# Exporting images

You can export images from Earth Engine in [GeoTIFF](https://trac.osgeo.org/geotiff/) or [TFRecord](https://www.tensorflow.org/api_docs/python/tf/io#TFRecords_Format_Details) format. See [Configuration Parameters](https://developers.google.com/earth-engine/exporting#configuration-parameters) for more output options.


In [ ]:
# Load a landsat image and select three bands.
landsat = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_123032_20140515')\
            .select(['B4', 'B3', 'B2'])

# Create a geometry representing an export region.
geometry = ee.Geometry.Rectangle([116.2621, 39.8412, 116.4849, 40.01236])
center = geometry.centroid().getInfo()['coordinates']
center.reverse()

Mapdisplay(center,{'landsat':landsat.getMapId()},zoom_start=7)

## to Drive

To export an image to your Drive account, use `Export.image.toDrive()`. For example, to export portions of a Landsat image, define a region to export, then call `ee.batch.Export.image.toDrive`:

In [ ]:
# Export the image, specifying scale and region.
task = ee.batch.Export.image.toDrive(**{
    'image': landsat,
    'description': 'imageToDriveExample',
    'folder':'Example_folder',
    'scale': 100,
    'region': geometry.getInfo()['coordinates']
})
task.start()

In [ ]:
import time 
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

## to Cloud Storage

To export an image to a Google Cloud Storage bucket, use 
`ee.batch.Export.image.toCloudStorage()`. To export the Landsat image in the previous example to Cloud Storage instead of Drive, use:

In [ ]:
outputBucket = 'bag_csaybar' #Change for your Cloud Storage bucket

# Export the image, specifying scale and region.
task = ee.batch.Export.image.toCloudStorage(**{
    'image': landsat,
      'description': 'imageToCOGeoTiffExample',
    'scale': 100,
    'region': geometry.getInfo()['coordinates'],
    'fileFormat': 'GeoTIFF',
    'bucket': outputBucket,
    'formatOptions': {'cloudOptimized': True}
})
task.start()

In [ ]:
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

## to Asset
To export an image to an asset in your Earth Engine assets folder, use `ee.batch.Export.image.toAsset()`. To manage your Earth Engine assets, or check how much of your storage quota is in use, use the [Asset Manager](https://developers.google.com/earth-engine/asset_manager). The following example illustrates exporting portions of a Landsat image using different pyramiding policies for the same band. The pyramiding policy indicates how Earth Engine computes lower-resolution versions of the asset. Learn more about how Earth Engine handles multiple resolutions in the [scale doc](https://developers.google.com/earth-engine/scale).

In [ ]:
# Get band 4 from the Landsat image, copy it.
band4 = landsat.select('B4').rename('b4_mean')\
               .addBands(landsat.select('B4').rename('b4_sample'))\
               .addBands(landsat.select('B4').rename('b4_max'))\

# Export the image to an Earth Engine asset.
task = ee.batch.Export.image.toAsset(**{
  'image': band4,
  'description': 'imageToAssetExample',
  'assetId': 'users/csaybar/exampleExport',
  'scale': 100,
  'region': geometry.getInfo()['coordinates']
})
task.start()

In [ ]:
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

You can provide a default pyramiding policy for every band that isn't explicitly specified by using the `'.default'` key. You may also pass in just the `'.default'` key. For example, to make all bands default to the 'sample' pyramiding policy, use `{'.default': 'sample'}`.

# Exporting tables and vector data

You can export a `FeatureCollection` as CSV, SHP (shapefile), GeoJSON, KML, KMZ or TFRecord using `Export.table`. The `FeatureCollection` may represent vectors or simply a table of data. In the latter case, the features in the collection will have null geometry.

Note some additional constraints when working with some file formats, including:

- KML: A `FeatureCollection` exported to a KML file will have all the geometries transformed to unprojected (WGS84) coordinates.

- SHP: A `FeatureCollection` exported to a Shapefile must contain features with the same geometry type and projection and must fit within the [Shapefile size limits](https://desktop.arcgis.com/en/arcmap/latest/manage-data/shapefiles/geoprocessing-considerations-for-shapefile-output.htm). Column names are truncated to 10 characters or fewer, and this must not create duplicate column names.

- TFRecord: A `FeatureCollection` exported to a TFRecord file will always be compressed with the GZIP compression type. You always get exactly one TFRecord file. All numbers are coerced to float type.

In [ ]:
# Make a collection of points.
features = ee.FeatureCollection([
  ee.Feature(ee.Geometry.Point(30.41, 59.933), {'name': 'Voronoi'}),
  ee.Feature(ee.Geometry.Point(-73.96, 40.781), {'name': 'Thiessen'}),
  ee.Feature(ee.Geometry.Point(6.4806, 50.8012), {'name': 'Dirichlet'})
])
center = features.geometry().centroid().getInfo()['coordinates']
center.reverse()
Mapdisplay(center,{'feature_example':features.getInfo()},zoom_start=2)

## to Drive
To export a FeatureCollection to your Drive account, use 
ee.batch.Export.table.toDrive(). For example:



In [ ]:
# Export the FeatureCollection to a KML file.
task = ee.batch.Export.table.toDrive(**{
  'collection': features,
  'description':'vectorsToDriveExample',
  'fileFormat': 'SHP'
})
task.start()

In [ ]:
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

## to Cloud Storage

To export a `FeatureCollection` to Cloud Storage, use ee.batch.Export.table.toCloudStorage(). For example, using the features defined previously:



In [ ]:
outputBucket = 'bag_csaybar' #Change for your Cloud Storage bucket

# Export a KML file to Cloud Storage.
task = ee.batch.Export.table.toCloudStorage(**{
  'collection': features,
  'description':'vectorsToCloudStorageExample',
  'bucket': outputBucket,
  'fileNamePrefix': 'exampleTableExport',
  'fileFormat': 'KML'    
})

task.start()

In [ ]:
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

## to Asset
To export a FeatureCollection as an Earth Engine asset, use `ee.batch.Export.table.toAsset()`. For example, using the features defined previously:

In [ ]:
# Export an ee.FeatureCollection as an Earth Engine asset.
task = ee.batch.Export.table.toAsset(**{
    'collection': features,
    'description':'exportToTableAssetExample',
    'assetId': 'users/csaybar/exampleExport'
})
task.start()

In [ ]:
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

There are several limitations on the size and shape of Earth Engine table assets:

- Maximum of 100 million features

- Maximum of 1000 properties (columns)

- Maximum of 100,000 vertices for each row's geometry

- Maximum of 100,000 characters per string value